In [2]:
from astroquery.skyview import SkyView
import matplotlib.pyplot as plt
import numpy as np
import astropy.units as u
from astropy.wcs import WCS
import matplotlib
from ipywidgets import widgets
from matplotlib import gridspec
from time import perf_counter
%matplotlib inline
matplotlib.rcParams['font.size']=15

In [3]:
surveys=['DSS2 IR', 'DSS2 Red','DSS2 Blue','WISE 22','GALEX Near UV','TGSS ADR1','NVSS']

In [4]:
img_dict={
    'Optical':[0, 1,2],
    'IOU' :  [3,1,4],
    'ROR' : [5, 1,6]
}

In [5]:
def make_RGB(RGB,survey ='Optical'):
    try:
        image=RGB[img_dict[survey]]
        image= np.stack((image[0],image[1],image[2]),axis=2)
    except :
        print(f"{survey} image not found for requested position")
        image=None
    return image

In [29]:
def RAD_RGB(position,radius,name,levels=4):
    try:
        paths_v = SkyView.get_images(position=position,pixels=600,scaling="Log",radius=radius*u.degree,survey=surveys[:-2])
        paths_r=SkyView.get_images(position=position,pixels=600,scaling="Sqrt",sampler='Lanczos3',radius=radius*u.degree,survey=surveys[-2:])
    except:    
        paths_v=[]
        for i in range(5):
            try:
                path = SkyView.get_images(position=position,pixels=600,scaling="Log",radius=radius*u.degree,survey=surveys[i])
                paths_v.append(path[0])
            except:
                display(widgets.Label(f"Data not found for {surveys[i]}"))
        paths_r=[]
        for i in range(5,7):
            try:
                path = SkyView.get_images(position=position,pixels=600,scaling="Sqrt",sampler='Lanczos3',radius=radius*u.degree,survey=surveys[i])
                paths_r.append(path[0])
            except:
                display(widgets.Label(f"Data not found for {surveys[i]}"))

    paths = paths_v + paths_r
  
    RGB=[]
    for path in paths:
        img=path[0].data.copy()

        min=img.min()
        img=img-min

        max=img.max()
        RGB.append(img/max)
    RGB_all=np.array(RGB)  

    contour=[]
    for path in paths_r:
        img=path[0].data.copy()
        contour.append(img)
    contour = np.array(contour)
    
    # RGB-TGSS
    RGB = make_RGB(RGB_all,survey='Optical')
    fig = plt.figure(constrained_layout=True,figsize=(15,15))
    gs  = gridspec.GridSpec(ncols=2, nrows=1, figure=fig)
    try:
        wcs = WCS(paths[0][0].header)
    except:
        wcs = None
    if wcs!=None:
        ax=fig.add_subplot(gs[0],projection=wcs)
        ax.imshow(RGB)
        ax.set_autoscale_on(False)

        img=contour[0]
        wcs = WCS(paths_r[0][0].header)
        levels_c=np.arange(0.015,img.max(),(img.max()-0.015)/levels)
        ax.contour(img, transform=ax.get_transform(wcs), colors='white',levels=levels_c)

        ax.set_title(f'RAD-RGB-Optical with TGSS of {position}',y=1,pad=-16,color="white")
        ax.annotate("#RADatHomeIndia",(10,10),color='white')
        ax.annotate(f"By {name}",(470-5*len(name),10),color='white')
        ax.axis("off")


    # IOU-TGSS
    RGB = make_RGB(RGB_all,survey='IOU')
    try:
        wcs = WCS(paths[3][0].header)
    except:
        wcs = None
    if wcs!=None:
        ax=fig.add_subplot(gs[1],projection=wcs)
        ax.imshow(RGB)

        ax.set_title(f'RAD-RGB-IOU with TGSS of  {position}',y=1,pad=-16,color='white')
        ax.annotate("#RADatHomeIndia",(10,10),color='white')
        ax.annotate(f"By {name}",(470-5*len(name),10),color='white')
        ax.set_autoscale_on(False)

        img=contour[0]
        wcs = WCS(paths_r[0][0].header)
        levels_c=np.arange(0.015,img.max(),(img.max()-0.015)/levels)
        ax.contour(img, transform=ax.get_transform(wcs), colors='white',levels=levels_c)
        ax.axis("off")
        plt.show()
      

    # ROR-TGSS
    fig = plt.figure(constrained_layout=True,figsize=(15,15))
    gs  = gridspec.GridSpec(ncols=2, nrows=1, figure=fig)
    
    RGB = make_RGB(RGB_all,survey='ROR')
    try:
        wcs = WCS(paths[6][0].header)
    except:
        wcs = None
    if wcs!=None:
        ax=fig.add_subplot(gs[0],projection=wcs)
        ax.imshow(RGB)

        ax.set_title(f'RAD-RGB-ROR with TGSS of {position}',y=1,pad=-16,color="white")
        ax.annotate("#RADatHomeIndia",(10,10),color='white')
        ax.annotate(f"By {name}",(470-5*len(name),10),color='white')
        ax.set_autoscale_on(False)

        img=contour[0]
        wcs = WCS(paths_r[0][0].header)
        levels_c=np.arange(0.015,img.max(),(img.max()-0.015)/levels)
        ax.contour(img, transform=ax.get_transform(wcs), colors='white',levels=levels_c)
        ax.axis("off")
    

    # ROR-NVSS
    RGB = make_RGB(RGB_all,survey='ROR')
    try:
        wcs = WCS(paths[6][0].header)
    except:
        wcs = None
    if wcs!=None:
        ax = fig.add_subplot(gs[1],projection=wcs)
        ax.imshow(RGB)

        ax.set_title(f'RAD-RGB-ROR with NVSS of {position}',y=1,pad=-16,color="white")
        ax.annotate("#RADatHomeIndia",(10,10),color='white')
        ax.annotate(f"By {name}",(470-5*len(name),10),color='white')
        ax.set_autoscale_on(False)

        img=contour[1]
        wcs = WCS(paths_r[1][0].header)
        levels_c=np.arange(0.0015,img.max(),(img.max()-0.0015)/levels)
        ax.contour(img, transform=ax.get_transform(wcs), colors='white',levels=levels_c)
        ax.axis("off")
        plt.show()

In [7]:
file = open("flogo.png", "rb")
image = file.read()
widgets.Image(
    value=image,
    format='png',
    width=300,
    height=400,
)

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x02\x12\x00\x00\x00\x9f\x08\x06\x00\x00\x00\xac\x8c0…


# **RGB Maker**

In [27]:
l1= widgets.Label("Your name ")
display(l1)
name = widgets.Text()
display(name)

l2= widgets.Label("Source name or coordinates(Eg. 06h 03m 20s 42 18 00)")
display(l2)
pos =widgets.Text()
display(pos)

l3= widgets.Label("Radius in degrees ")
display(l3)
r = widgets.Text()
display(r)
btn=widgets.Button(description="Submit")
display(btn)

output = widgets.Output()

@output.capture(clear_output=True,wait=True)

def submit(b):
    l4= widgets.Label("Loading images...",font_size=75)
    display(l4)
    start=perf_counter()
    try:
        if len(name.value)> 2 and len(pos.value)>2 and len(r.value)>0:
            RAD_RGB(position= pos.value,name=name.value,radius=float(r.value))
            time_taken=perf_counter()-start
            l6= widgets.Label(f"Image loading finished, Time elapsed{time_taken}")
            display(l6)
        elif len(r.value)> 0 and len(pos.value) >2:
            l5= widgets.Label("Default name set")
            display(l5)
            RAD_RGB(position= pos.value,name="Anonymous",radius=float(r.value)) 
            time_taken=perf_counter()-start
            l6= widgets.Label(f"Image loading finished. Time elapsed {np.round (time_taken,3)} seconds")
            display(l6)
            #link = widgets.HTML(value="<font color='blue'><a href=https://l.facebook.com/l.php?u=https%3A%2F%2Fdocs.google.com%2Fdocument%2Fd%2F1U5nkmCKOlgnDk4pO7d7HKj_OidH9IvXNfW9AojBc-s0%2Fedit%3Fusp%3Dsharing%26fbclid%3DIwAR2QrMEHyHCmmfuBOB5_yxDeqPVN1IrKeWY_iaQqn6HJI7IMjtEw9nJChBk&h=AT0S_OeZAN4e_db03mDydoXvBpoqB9C2iDkM1Uwmedg_bfickr0cVzdc7bwgWV3XX475c6aGRnNTy0l84e0gMBO2pM0CjKL8-ogb9OHBq0Z43gaPkshjRVrLXJnZfAcN3Fi1zQ>RAD-RGB image Acknowledgement</a>" )
            #display(link)
        else:
            l5= widgets.Label("Enter Position and Radius")
            display(l5)
    except:
        l5= widgets.Label("Data not Found")
        display(l5)       
btn.on_click(submit)
output

Label(value='Your name ')

Text(value='')

Label(value='Source name or coordinates(Eg. 06h 03m 20s 42 18 00)')

Text(value='')

Label(value='Radius in degrees ')

Text(value='')

Button(description='Submit', style=ButtonStyle())

Output()

In [25]:
foot=widgets.HTML(value="<style> #footer { position: fixed; padding: 10px 10px 0px 10px; bottom: 0; width: 94.5%;  /* Height of the footer*/  height: 40px; background: black; } </style><center> <div id=\"footer\"><a href=https://l.facebook.com/l.php?u=https%3A%2F%2Fdocs.google.com%2Fdocument%2Fd%2F1U5nkmCKOlgnDk4pO7d7HKj_OidH9IvXNfW9AojBc-s0%2Fedit%3Fusp%3Dsharing%26fbclid%3DIwAR2QrMEHyHCmmfuBOB5_yxDeqPVN1IrKeWY_iaQqn6HJI7IMjtEw9nJChBk&h=AT0S_OeZAN4e_db03mDydoXvBpoqB9C2iDkM1Uwmedg_bfickr0cVzdc7bwgWV3XX475c6aGRnNTy0l84e0gMBO2pM0CjKL8-ogb9OHBq0Z43gaPkshjRVrLXJnZfAcN3Fi1zQ><font color='blue'><u>RAD-RGB image Acknowledgement</u></a></div></center> ")
display(foot)

HTML(value='<style> #footer { position: fixed; padding: 10px 10px 0px 10px; bottom: 0; width: 94.5%;  /* Heigh…